Задание 1
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [1]:
import pandas as pd

In [133]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
movies = pd.read_csv('ml-latest-small/movies.csv')

In [6]:
def average_rating (movie):
    return movie.rating.mean()

In [44]:
average_ratings = ratings.groupby('movieId').apply(average_rating).reset_index()
average_ratings.columns = ['movieId', 'av_rating']

In [45]:
def sort_movies (av_rating):
    if av_rating <= 2:
        movie_class = "low grade"
    elif av_rating <= 4:
        movie_class = "middle grade"
    elif av_rating == 4.5 or av_rating == 5:
        movie_class = "high grade"
    else:
        movie_class = "grade not defined"   
    return movie_class

In [47]:
average_ratings['class'] = average_ratings['av_rating'].apply(sort_movies)

In [49]:
movies_rating = movies.merge(average_ratings[['movieId', 'class']], on = 'movieId')

In [58]:
movies_rating.head()

,movieId,title,genres,class
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,middle grade
1,2,Jumanji (1995),Adventure|Children|Fantasy,middle grade
2,3,Grumpier Old Men (1995),Comedy|Romance,middle grade
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,middle grade
4,5,Father of the Bride Part II (1995),Comedy,middle grade


Задание 2
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [174]:
keywords = pd.read_csv('ml-latest-small/keywords.csv')

In [72]:
geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

In [78]:
inv_geo = {}
for region in geo_data.keys():
    inv_geo_reg = {city: region for city in geo_data[region]}
    inv_geo.update(inv_geo_reg)

In [120]:
def find_region(request):
    for city in list(inv_geo.keys()):
        if city in request:
            key_region = inv_geo[city]
            break
        else: 
            key_region = 'undefined'
    return key_region

In [124]:
keywords['region'] = keywords['keyword'].apply(find_region)

In [131]:
keywords[ keywords['keyword'].str.contains('хабаровск', case=False) ].head()

,keyword,shows,region
2175,дром хабаровск,98361,Дальний Восток
3687,авито хабаровск,63736,Дальний Восток
6260,фарпост хабаровск,40962,Дальний Восток
8039,погода в хабаровске,33437,Дальний Восток
9084,хабаровск,29655,Дальний Восток


In [132]:
keywords.head()

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined


Задание 3 (бонусное)

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл movies.csv и ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

В переменную years запишите список из всех годов с 1950 по 2010.

Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

для каждой строки пройдите по всем годам списка years
если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [144]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
movies = pd.read_csv('ml-latest-small/movies.csv')
years = [str(y) for y in range(1950, 2011)]

In [152]:
def production_year(film_title):
    for year in years:
        if year in film_title:
            film_year = int(year)
            break
        else: 
            film_year = 1900
    return film_year

In [153]:
movies['year'] = movies['title'].apply(production_year)

In [155]:
ratings_years = ratings.merge(movies[['movieId', 'year']], on = 'movieId')

In [167]:
def average_year_rating (year):
    return year.rating.mean()

In [170]:
average_rating_year = ratings_years.groupby('year').apply(average_year_rating).reset_index()
average_rating_year.columns = ['year', 'rating']

In [173]:
average_rating_year.sort_values(by = 'rating', ascending = False)

,year,rating
8,1957,4.014241
23,1972,4.011136
3,1952,4.000000
5,1954,3.994220
2,1951,3.983539
...,...,...
56,2005,3.448434
54,2003,3.444777
47,1996,3.426600
48,1997,3.415764
